In [43]:
import csv
import pandas as pd
import numpy as np
from nltk import tokenize
from scipy import spatial
from tqdm import tqdm

In [2]:
path = "/home/cambirdgepapa/data/pf.csv"
df = pd.read_csv(path)

In [3]:
df.head()

,id,artist,album,genre,score,date,author,role,review,bnm,link
0,0,David Byrne,“…The Best Live Show of All Time” — NME EP,Rock,5.5,January 11 2019,Andy Beta,Contributor,"Viva Brother, Terris, Mansun, the Twang, Joe L...",0,https://pitchfork.com/reviews/albums/david-byr...
1,1,DJ Healer,Lost Lovesongs / Lostsongs Vol. 2,Electronic,6.2,January 11 2019,Chal Ravens,Contributor,"The Prince of Denmark—that is, the proper prin...",0,https://pitchfork.com/reviews/albums/dj-healer...
2,2,Jorge Velez,Roman Birds,Electronic,7.9,January 10 2019,Philip Sherburne,Contributing Editor,"Jorge Velez has long been prolific, but that’s...",0,https://pitchfork.com/reviews/albums/jorge-vel...
3,3,Chandra,Transportation EPs,Rock,7.8,January 10 2019,Andy Beta,Contributor,When the Avalanches returned in 2016 after an ...,0,https://pitchfork.com/reviews/albums/chandra-t...
4,4,The Chainsmokers,Sick Boy,Electronic,3.1,January 9 2019,Larry Fitzmaurice,Contributor,We’re going to be stuck with the Chainsmokers ...,0,https://pitchfork.com/reviews/albums/the-chain...


In [4]:
# construct graph
# if review similarity > xxx, create an edge

In [5]:
df.shape

(20873, 11)

In [4]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('bert-base-nli-mean-tokens')

100%|██████████| 405M/405M [00:48<00:00, 8.33MB/s] 


In [108]:
df_rhp = df[df["artist"]=="Red House Painters"]
df_skm = df[df["artist"]=="Sun Kil Moon"]
df_kl = df[df["artist"]=="Kendrick Lamar"]
df_weezer = df[df["artist"]=="Weezer"]

In [109]:
df_weezer.head()

,id,artist,album,genre,score,date,author,role,review,bnm,link
1424,1424,Weezer,Pacific Daydream,Rock,4.3,October 26 2017,Saby Reyes-Kulkarni,Contributor,"Long ago, Rivers Cuomo basically split Weezer ...",0,https://pitchfork.com/reviews/albums/weezer-pa...
2288,2291,Weezer,Weezer (Blue Album),Rock,10.0,February 26 2017,Jillian Mapes,Senior Editor,Weezer mastermind Rivers Cuomo was such a somb...,0,https://pitchfork.com/reviews/albums/22858-wee...
3436,3441,Weezer,Weezer (White Album),Rock,6.2,March 30 2016,Zoe Camp,Contributor,"In 2014, amidst the wreckage of a long, dispir...",0,https://pitchfork.com/reviews/albums/21593-wee...
5121,5132,Weezer,Everything Will Be Alright in the End,Rock,6.5,October 2 2014,Ian Cohen,Contributor,Rivers Cuomo has taken your shit for over two ...,0,https://pitchfork.com/reviews/albums/19830-wee...
9719,9734,Weezer,Pinkerton [Deluxe Edition],Rock,10.0,November 3 2010,Ian Cohen,Contributor,"At one point, there seemed as strong a chance ...",1,https://pitchfork.com/reviews/albums/14817-wee...


In [66]:
emb1 = np.mean(model.encode(tokenize.sent_tokenize(df_rhp["review"].values[0])[:30]),0)
emb2 = np.mean(model.encode(tokenize.sent_tokenize(df_rhp["review"].values[1])[:30]),0)
emb3 = np.mean(model.encode(tokenize.sent_tokenize(df_skm["review"].values[1])[:30]),0)
emb4 = np.mean(model.encode(tokenize.sent_tokenize(df_skm["review"].values[1])[:30]),0)

In [50]:
def cosine(x,y):
    return 1 - spatial.distance.cosine(x, y)

In [68]:
print (cosine(emb1, emb2) , cosine(emb1, emb3), cosine(emb2, emb3),cosine(emb1, emb4))

0.8771730661392212 0.8797327876091003 0.8805571794509888 0.8797327876091003


In [47]:
print (emb2.shape, len(emb2))

(768,) 768


["Red House Painters were fairly popular in the early '90s, but details on them were scarce.",
 'This was all before the Internet, so you basically had what a jewel box told you.',
 "It's hard to imagine at this point, but when I first heard them, I didn't know anything about Mark Kozelek, the frontman and mastermind behind the project.",
 "I'm not sure I knew his name, and I definitely didn't know what he looked like, or his personality outside of the songs.",
 "I also had no idea the band was a quartet, or that they lived in San Francisco (in Kozelek's case, via Ohio).",
 'At least for me, Red House Painters existed outside of a scene, too.',
 'They had a familiar record label with its own distinct aesthetic, but that was the only real touchstone.',
 'Not that any of this is bad.',
 "The songs on the first four albums—1992's Down Colorful Hill, 1993's pair of self-titled albums (the first nicknamed Rollercoaster, the second\xa0Bridge, after the photographs on their covers), and 1995'

In [5]:
# TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.datasets import fetch_20newsgroups

In [6]:
tfidf_review = TfidfVectorizer().fit_transform(df["review"].values.astype('U'))

In [7]:
print (tfidf_review.shape)

(20873, 131154)


In [8]:
from sklearn.metrics.pairwise import linear_kernel

In [9]:
print (linear_kernel(tfidf_review[2100], tfidf_review[3520]).flatten())
print (linear_kernel(tfidf_review[2100], tfidf_review[8890]).flatten())
print (linear_kernel(tfidf_review[2100], tfidf_review[2000]).flatten())
print (linear_kernel(tfidf_review[2279], tfidf_review[128]).flatten()) # same band
print (linear_kernel(tfidf_review[2279], tfidf_review[8890]).flatten())
print (linear_kernel(tfidf_review[2279], tfidf_review[4471]).flatten()) # same genre
print (linear_kernel(tfidf_review[7428], tfidf_review[8890]).flatten()) # same artist
print (linear_kernel(tfidf_review[15610], tfidf_review[17858]).flatten()) # same band
print (linear_kernel(tfidf_review[14100], tfidf_review[17858]).flatten()) # same band
print (linear_kernel(tfidf_review[4471], tfidf_review[20184]).flatten()) # same band
print (linear_kernel(tfidf_review[1424], tfidf_review[20184]).flatten()) # same band

[0.38483497]
[0.41506844]
[0.14346324]
[0.65080653]
[0.18875779]
[0.60217988]
[0.45245446]
[0.44177944]
[0.56380393]
[0.52774403]
[0.2358361]


In [10]:
pairwise_sim = linear_kernel(tfidf_review, tfidf_review)

In [11]:
# normalize pairwise_sim
#pairwise_sim_std = (pairwise_sim - np.mean(pairwise_sim)) / np.std(pairwise_sim)

In [19]:
pairwise_sim_no_diag = pairwise_sim - np.eye(pairwise_sim.shape[0]) * pairwise_sim

In [75]:
# creat edges if >th
indices0_35 = np.argwhere(pairwise_sim_no_diag>0.35)
indices0_4 = np.argwhere(pairwise_sim_no_diag>0.4)
indices0_45 = np.argwhere(pairwise_sim_no_diag>0.45)
indices0_5 = np.argwhere(pairwise_sim_no_diag>0.5)

In [76]:
print (len(indices0_35),len(indices0_4),len(indices0_45),len(indices0_5))

734412 125260 41356 19502


In [ ]:
recorded = []
recorded_ = []
for ind in tqdm(indices0_35):
    r = str(ind[0])+" "+str(ind[1])
    rr = str(ind[1])+" "+str(ind[0])
    if (r not in recorded) and (rr not in recorded):
        #relations_ind_.append(r+" "+str(sim.item()))
        recorded.append(r)
        sim = pairwise_sim_no_diag[ind[0]][ind[1]]
        recorded_.append(r+" "+str(float(sim.item())))

In [ ]:
with open('th0_35_weighted.txt', 'w') as f:
    for item in recorded_:
        f.write("%s\n" % item)    

In [53]:
df_meta = df[["album","artist","genre","score","date","link"]]

In [54]:
df_meta

,album,artist,genre,score,date,link
0,“…The Best Live Show of All Time” — NME EP,David Byrne,Rock,5.5,January 11 2019,https://pitchfork.com/reviews/albums/david-byr...
1,Lost Lovesongs / Lostsongs Vol. 2,DJ Healer,Electronic,6.2,January 11 2019,https://pitchfork.com/reviews/albums/dj-healer...
2,Roman Birds,Jorge Velez,Electronic,7.9,January 10 2019,https://pitchfork.com/reviews/albums/jorge-vel...
3,Transportation EPs,Chandra,Rock,7.8,January 10 2019,https://pitchfork.com/reviews/albums/chandra-t...
4,Sick Boy,The Chainsmokers,Electronic,3.1,January 9 2019,https://pitchfork.com/reviews/albums/the-chain...
...,...,...,...,...,...,...
20868,Intimate Lightning,Neptune,"Experimental,Rock",7.9,June 20 2004,https://pitchfork.com/reviews/albums/5901-inti...
20869,We Are from Nowhere,Dance Disaster Movement,Experimental,6.5,June 1 2003,https://pitchfork.com/reviews/albums/2160-we-a...
20870,Failer,Kathleen Edwards,Rock,7.5,May 29 2003,https://pitchfork.com/reviews/albums/2684-failer/
20871,Reservoir Songs EP,Crooked Fingers,Rock,8.0,May 22 2002,https://pitchfork.com/reviews/albums/1669-rese...
